<a href="https://colab.research.google.com/github/valeromora/TAM_2025-1/blob/main/Final%20Proyect/Final_proyect_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up

In [1]:
# Install necessary libraries if they are not already installed
!pip install PyDrive

In [2]:
!pip install nltk

In [3]:
!pip install -U sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Database

In [5]:
# https://drive.google.com/file/d/1x6HASgMM9WqaglFR-zH2_LU6eRs4l_tq/view?usp=sharing

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download the file
file_id = '1x6HASgMM9WqaglFR-zH2_LU6eRs4l_tq'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('downloaded_file.zip')

# Unzip the file
with zipfile.ZipFile('downloaded_file.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/')

# List the extracted files to confirm
!ls /content/

'12EE54A_6.2.1 - Safely managed sanitation & hand-washing'
'1548EA3_6.1.1 - Safely managed drinking water'
'16BBF41_3.4.2 - Suicide'
'1772666_3.1.2 - Births attended by skilled health personnel'
'1F96863_3.4.1 - cardiovascular disease, cancer, diabetes or chronic respiratory disease'
'217795A_3.C.1 - Health worker density and distribution'
'2322814_3.2.1 - Under-five mortality rate'
'2D6FBE4_3.3.5 - Neglected tropical diseases'
'361734E_16.1.1 - Intentional homicide'
'442CEA8_3.3.3 - Malaria'
'45CA7C8_3.C.1 - Health worker density and distribution'
'5C8435F_3.C.1 - Health worker density and distribution'
'5F8A486_2.2.1 - Stunting'
 608DE39
'6A64C9A_7.1.2 - Clean fuels'
'75DDA77_3.A.1 - Age-standardized prevalence of tobacco use'
'77D059C_3.3.1 - HIV infections'
'8074BD9_3.7.1 - Women satisfied with modern methods'
'84FD3DE_3.9.3 - Unintentional poisoning'
'A37BDD6_6.3.1 - Safely treated wastewater'
'A4C49D3_3.2.2- Neonatal mortality rate'
'AC597B1_3.1.1 - Maternal mortality ratio'
'B9C

# Preparación archivos
### Extracción y enriquecimiento de información por carpeta
Este bloque de código recorre cada carpeta del dataset, busca archivos relevantes (`Metadata`, `Dictionary`, `Code list`, `Dataset`) y construye documentos enriquecidos en texto plano.

- Se extraen definiciones de variables desde el archivo **Data Dictionary**.
- Se interpretan valores codificados usando el **Code List** (si está presente).
- Se incluye el contexto general desde el archivo **Metadata**.
- Cada fila del Dataset es enriquecida con descripciones legibles para facilitar su comprensión y futura vectorización.
- El resultado se almacena en una lista de diccionarios llamada `documents`, lista para dividirse en _chunks_.

Esto permite aprovechar mejor los datos, incluso si vienen organizados de manera heterogénea.


In [6]:
base_path = "/content"
csv_count = 0
folders_with_csv = {}

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
        csv_count += len(csv_files)
        folders_with_csv[folder] = csv_files

print(f"Total de archivos .csv encontrados: {csv_count}")
print(f"Resumen por carpeta:")
for folder, files in folders_with_csv.items():
    print(f"{folder}: {len(files)} archivos")


Total de archivos .csv encontrados: 139
Resumen por carpeta:
.config: 0 archivos
1772666_3.1.2 - Births attended by skilled health personnel: 3 archivos
F810947_11.6.2 - Fine particulate matter: 4 archivos
AC597B1_3.1.1 - Maternal mortality ratio: 3 archivos
84FD3DE_3.9.3 - Unintentional poisoning: 4 archivos
BEFA58B: 4 archivos
F513188_3.3.4 - Hepatitis B: 3 archivos
B9C6C79_1.a.2 - Government spending essential services: 3 archivos
12EE54A_6.2.1 - Safely managed sanitation & hand-washing: 4 archivos
2322814_3.2.1 - Under-five mortality rate: 4 archivos
361734E_16.1.1 - Intentional homicide: 4 archivos
D1223E8_6.2.1 - Safely managed sanitation & hand-washing: 4 archivos
6A64C9A_7.1.2 - Clean fuels: 4 archivos
EF93DDB: 4 archivos
5F8A486_2.2.1 - Stunting: 3 archivos
C288D13_3.3.2 - Tuberculosis: 3 archivos
A37BDD6_6.3.1 - Safely treated wastewater: 3 archivos
A4C49D3_3.2.2- Neonatal mortality rate: 4 archivos
F8524F2_5.2.1 - Intimate partner violence (last 12 months): 3 archivos
D6176E

In [7]:
# Ruta base donde se encuentran las carpetas con los datasets y archivos relacionados
base_path = "/content"

# Lista que almacenará los documentos generados por cada carpeta,
# donde cada documento es un texto enriquecido con metadata, definiciones y datos
documents = []

# Recorremos cada carpeta dentro de la ruta base
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    # Saltamos si no es carpeta (p.ej., archivos sueltos)
    if not os.path.isdir(folder_path):
        continue

    # Diccionarios de mapeo para traducir y dar contexto:
    # data_dict_map: mapea nombre de variable → definición textual
    data_dict_map = {}
    # code_list_map: mapea (dimensión, código) → descripción del código
    code_list_map = {}

    # Variable para acumular el texto de metadata (solo un archivo por carpeta)
    metadata_text = ""

    # Búsqueda y extracción del archivo de Metadata para la carpeta actual
    # Solo se procesa el primer archivo que cumpla la condición
    for filename in os.listdir(folder_path):
        if "Metadata" in filename and filename.endswith(".csv"):
            try:
                df_meta = pd.read_csv(os.path.join(folder_path, filename))
                metadata_text += f"📄 Metadata de {filename}:\n"
                # Recorremos las filas sin índice para extraer clave y valor
                for row in df_meta.itertuples(index=False):
                    if len(row) >= 2:
                        key = str(row[0]).strip()
                        value = str(row[1]).strip()
                        metadata_text += f"{key}: {value}\n"
            except Exception as e:
                print(f"No se pudo leer metadata en {folder}: {e}")
            break  # Solo procesamos un archivo Metadata por carpeta

    # Primera pasada: cargamos diccionarios para darle sentido a los datos
    for filename in os.listdir(folder_path):
        # Procesamos solo archivos CSV
        if not filename.endswith(".csv"):
            continue

        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"No se pudo leer {filename}: {e}")
            continue

        # Si es un archivo Data Dictionary, cargamos definiciones de variables
        if "Dictionary" in filename or "Data Dictionary" in filename:
            for row in df.itertuples(index=False):
                if len(row) >= 2:
                    var = str(row[0]).strip()
                    definition = str(row[1]).strip()
                    # Guardamos mapeo: nombre_variable -> definición
                    data_dict_map[var] = definition

        # Si es un archivo Code List, cargamos descripciones de códigos por dimensión
        elif "Code list" in filename:
            for row in df.itertuples(index=False):
                if len(row) >= 4:
                    dim = str(row[0]).strip()     # Dimensión o categoría
                    key = str(row[1]).strip()     # Código o clave
                    name = str(row[2]).strip()    # Nombre o etiqueta
                    desc = str(row[3]).strip()    # Descripción detallada
                    # Guardamos mapeo: (dimensión, código) -> descripción enriquecida
                    code_list_map[(dim, key)] = f"{name}: {desc}"

    # Segunda pasada: procesamos archivos que contengan los datos en sí
    for filename in os.listdir(folder_path):
        # Saltamos si no es CSV o si es Dictionary o Code list (ya procesados)
        if not filename.endswith(".csv") or "Dictionary" in filename or "Code list" in filename:
            continue

        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"No se pudo leer {filename}: {e}")
            continue

        text_content = ""

        # Si por algún motivo volvemos a encontrar metadata, la agregamos
        if "Metadata" in filename:
            text_content += f"🧾 Metadata extraída de {filename}:\n"
            for row in df.itertuples(index=False):
                if len(row) >= 2:
                    key = str(row[0]).strip()
                    value = str(row[1]).strip()
                    text_content += f"{key}: {value}\n"

        # Si es un dataset con datos, construimos textos enriquecidos
        elif "Dataset" in filename:
            text_content += f"📊 Datos enriquecidos de {filename}:\n"
            # Recorremos fila por fila
            for row in df.itertuples(index=False):
                row_description = []
                # Para cada columna, buscamos su definición y posible descripción de valor
                for col in df.columns:
                    # Valor en la celda actual
                    value = getattr(row, col, "")
                    # Nombre legible para la variable, si existe
                    label = data_dict_map.get(col, col)

                    # Si el valor es una cadena y existe descripción en code_list para esa dimensión y valor
                    if isinstance(value, str) and (col, value) in code_list_map:
                        value_desc = code_list_map[(col, value)]
                        row_description.append(f"{label}: {value_desc}")
                    else:
                        # Si no hay descripción especial, usamos el valor tal cual
                        row_description.append(f"{label}: {value}")

                # Concatenamos la descripción enriquecida para la fila completa
                text_content += " | ".join(row_description) + "\n"

        else:
            # Archivos no reconocidos se reportan para control
            print(f"Archivo sin tipo conocido: {filename}")
            continue

        # Finalmente, agregamos a la lista de documentos con su fuente y contenido enriquecido
        documents.append({
            "source": f"{folder}/{filename}",
            "content": metadata_text + "\n" + text_content
        })

Archivo sin tipo conocido: mnist_test.csv
Archivo sin tipo conocido: california_housing_train.csv
Archivo sin tipo conocido: california_housing_test.csv
Archivo sin tipo conocido: mnist_train_small.csv


In [8]:
print(f"Total documentos extraídos: {len(documents)}")
print(documents[0]['source'])
print(documents[0]['content'][:1000])  # muestra los primeros caracteres del primer documento

Total documentos extraídos: 76
1772666_3.1.2 - Births attended by skilled health personnel/170_1772666_Metadata_2025-04-07.csv
📄 Metadata de 170_1772666_Metadata_2025-04-07.csv:
Name: Proportion of births attended by skilled health personnel (%)
Short name: Proportion of births attended by skilled health personnel (%)
Indicator unique identifier: 1772666
Indicator codes: MDG_0000000025
Also known as: SDG indicator 3.1.2
SDG Goal: 3.1.2 – Births attended by skilled health personnel
Short description: Proportion of births attended by skilled health personnel.
(SDG 3.1.2)
Definition: Proportion of births attended by skilled health personnel (generally doctors, nurses or midwives but can refer to other health professionals providing childbirth care) is the proportion of childbirths attended by professional health personnel.
According to the current definition (1) these are competent maternal and newborn health (MNH) professionals educated, trained and regulated to national and internationa

# Chunking

Se busca dividir cada content del documento en trozos más pequeños.
* Divide el texto en grupos de 500 palabras

* Repite las últimas 50 palabras del chunk anterior (para mantener contexto)

In [22]:
MAX_CHUNK_WORDS = 500
OVERLAP = 50

chunks = []

for doc in documents:
    source = doc["source"]
    words = doc["content"].split()

    for i in range(0, len(words), MAX_CHUNK_WORDS - OVERLAP):
        chunk_words = words[i:i + MAX_CHUNK_WORDS]
        chunk_text = ' '.join(chunk_words)

        chunks.append({
            "source": source,
            "content": chunk_text
        })

In [23]:
print(f"Total de chunks: {len(chunks)}")
print("Ejemplo de chunk:\n", chunks[0]["content"][:700])

Total de chunks: 576
Ejemplo de chunk:
 📄 Metadata de 170_1772666_Metadata_2025-04-07.csv: Name: Proportion of births attended by skilled health personnel (%) Short name: Proportion of births attended by skilled health personnel (%) Indicator unique identifier: 1772666 Indicator codes: MDG_0000000025 Also known as: SDG indicator 3.1.2 SDG Goal: 3.1.2 – Births attended by skilled health personnel Short description: Proportion of births attended by skilled health personnel. (SDG 3.1.2) Definition: Proportion of births attended by skilled health personnel (generally doctors, nurses or midwives but can refer to other health professionals providing childbirth care) is the proportion of childbirths attended by professional health person


# Vectorización (embeddings)
Los modelos de lenguaje como los usados en RAG no buscan respuestas de forma exacta por coincidencia de palabras, sino que comparan significados. Para hacer esto, necesitamos representar cada fragmento de texto (chunk) como un vector en un espacio multidimensional. Luego podemos encontrar los textos más relevantes comparando su distancia semántica.

Utilizamos el modelo all-MiniLM-L6-v2 de la librería sentence-transformers, el cual transforma cada chunk de texto en un vector numérico de 384 dimensiones que resume su contenido de forma semántica. Luego extraemos los textos de los chunks previamente generados y los vectorizamos

In [11]:
# Cargar el modelo
#model = SentenceTransformer('all-MiniLM-L6-v2')

# Extraer solo el texto de los chunks
#texts = [chunk['content'] for chunk in chunks]

# Vectorizar los textos
#embeddings = model.encode(texts, show_progress_bar=True)

# Convertir a matriz numpy
#embeddings_np = np.array(embeddings)

In [16]:
import os
import google.generativeai as genai
import pandas as pd
import numpy as np # Asegurarse de importar numpy

# Configura tu clave de API de Gemini.
# Es mejor usar Colab Secrets para almacenar tu clave de API de forma segura.
# Ve a "🔑" en el panel de la izquierda, agrega un nuevo secreto llamado "GEMINI_API_KEY"
# y pega tu clave de API allí. Luego, usa la siguiente línea para acceder a ella:
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Esta función toma una oración como argumento y devuelve sus embeddings
def get_embeddings(text):
    # Define the embedding model
    model = 'models/embedding-001'
    # Get the embeddings
    embedding = genai.embed_content(model=model,
                                    content=text,
                                    task_type="retrieval_document")
    return embedding['embedding']

# Asegúrate de que 'chunks' esté definido y contenga tus documentos (chunks)
# Por ejemplo, si usaste la variable 'chunks' en celdas anteriores:
if 'chunks' not in globals():
    print("La variable 'chunks' no está definida. Asegúrate de haber ejecutado las celdas anteriores para crear los chunks.")
else:
    # Get the content from the chunks and create a new list
    content_list = [chunk['content'] for chunk in chunks]

    # Send one chunk content at a time to get embeddings
    print(f"Generando embeddings para {len(content_list)} chunks...")
    # Implementación de un manejo de errores básico y reintentos para la API de Gemini
    embeddings = []
    for i, content in enumerate(content_list):
        try:
            embeddings.append(get_embeddings(content))
            if (i + 1) % 10 == 0:
                print(f"Embeddings generados para {i + 1}/{len(content_list)} chunks.")
        except Exception as e:
            print(f"Error al generar embedding para chunk {i}: {e}. Saltando este chunk.")
            # Puedes optar por agregar un embedding de ceros o None si un chunk falla
            # embeddings.append([0.0] * dimension) # Asumiendo que conoces la dimensión
            # O simplemente saltarlo, lo que hará que la lista de embeddings sea más corta que content_list

    print("Generación de embeddings finalizada.")

    # Convertir a matriz numpy
    # Asegurarse de que todos los embeddings tengan la misma longitud antes de convertir a numpy
    if embeddings:
        # Puedes verificar la longitud del primer embedding para la dimensión
        # dimension = len(embeddings[0])
        # O si usaste SentenceTransformer antes, podrías usar la dimensión de ese modelo (768)
        # dimension = 768 # O la dimensión correcta de 'models/embedding-001'

        # Convertir a numpy array
        embeddings_np_gemini = np.array(embeddings)
        print("Embeddings de Gemini convertidos a numpy array.")
        print("Forma del array de embeddings de Gemini:", embeddings_np_gemini.shape)

        # Nota: Este array de embeddings (embeddings_np_gemini) es diferente al generado por SentenceTransformer.
        # Deberás decidir cuál usar para construir tu índice FAISS.
        # Si quieres usar estos embeddings de Gemini con el índice FAISS existente,
        # necesitarías crear un nuevo índice FAISS con la dimensión correcta para estos embeddings
        # y añadir este nuevo array a ese índice.
    else:
        print("No se generaron embeddings.")

Generando embeddings para 576 chunks...
Embeddings generados para 10/576 chunks.
Error al generar embedding para chunk 12: Failed to parse JSON: TypeError: 'NoneType' object is not iterable.. Saltando este chunk.
Error al generar embedding para chunk 18: Failed to parse JSON: TypeError: 'NoneType' object is not iterable.. Saltando este chunk.
Embeddings generados para 20/576 chunks.
Embeddings generados para 30/576 chunks.
Embeddings generados para 40/576 chunks.
Embeddings generados para 50/576 chunks.
Embeddings generados para 60/576 chunks.
Embeddings generados para 70/576 chunks.
Embeddings generados para 80/576 chunks.
Embeddings generados para 90/576 chunks.
Embeddings generados para 100/576 chunks.
Embeddings generados para 110/576 chunks.
Embeddings generados para 120/576 chunks.
Embeddings generados para 130/576 chunks.
Embeddings generados para 140/576 chunks.
Embeddings generados para 150/576 chunks.
Embeddings generados para 160/576 chunks.
Embeddings generados para 170/576

Usamos FAISS (Facebook AI Similarity Search) para almacenar los vectores y poder hacer búsquedas por similitud. Esto nos permite recuperar los chunks más cercanos a una consulta (query) en términos de significado.

In [17]:
# Creamos un índice FAISS para búsquedas por similitud
# La dimensión debe coincidir con la del array de embeddings de Gemini
if 'embeddings_np_gemini' in globals() and embeddings_np_gemini.shape[0] > 0:
    dimension = embeddings_np_gemini.shape[1]
    index = faiss.IndexFlatL2(dimension)

    # Añadir los vectores de Gemini
    index.add(embeddings_np_gemini)
    print(f"Índice FAISS creado con {index.ntotal} vectores de dimensión {dimension}.")
else:
    print("No se encontraron embeddings de Gemini válidos para crear el índice FAISS.")

Índice FAISS creado con 574 vectores de dimensión 768.


In [18]:
# Guardar los metadatos (puede ser útil para devolver respuestas)
# Asegurarse de que la lista de chunks sea coherente con los embeddings generados
if 'chunks' in globals() and 'embeddings_np_gemini' in globals() and len(chunks) >= embeddings_np_gemini.shape[0]:
    # Si algunos chunks fallaron al generar embeddings, solo guarda los metadatos para los chunks exitosos
    # Una forma simple es asumir que los primeros N chunks corresponden a los N embeddings exitosos
    # Sin embargo, si hubo saltos (como en la salida anterior), la correspondencia directa puede ser incorrecta.
    # Un enfoque más robusto sería mantener un registro de qué chunks generaron embeddings exitosamente.
    # Por ahora, asumiremos que el orden se mantuvo para los chunks exitosos y truncamos si es necesario.
    chunk_sources = [chunk["source"] for chunk in chunks[:embeddings_np_gemini.shape[0]]]
    print(f"Guardados metadatos para {len(chunk_sources)} chunks correspondientes a los embeddings generados.")
elif 'chunks' in globals():
     print("Advertencia: El número de chunks no coincide con el número de embeddings generados.")
     chunk_sources = [chunk["source"] for chunk in chunks]
else:
     print("No se encontraron chunks para guardar metadatos.")

Guardados metadatos para 574 chunks correspondientes a los embeddings generados.


Con el índice creado, ahora podemos consultar el sistema en lenguaje natural. FAISS devuelve los chunks más relevantes con base en la similitud semántica entre la consulta y los documentos vectorizados.

In [19]:
# Ejemplo de búsqueda
query = "malaria"
# Vectorizar la consulta usando la función get_embeddings
query_vec = get_embeddings(query)

# Buscar los 5 chunks más similares. index.search espera numpy arrays.
# get_embeddings devuelve una lista, la convertimos a numpy array y ajustamos la forma si es necesario
query_vec_np = np.array([query_vec])


distances, indices = index.search(query_vec_np, k=5)

# Mostrar los resultados
print(f"Resultados de la búsqueda para '{query}':")
# Asegurarse de tener chunk_sources disponibles
if 'chunk_sources' in globals() and len(chunk_sources) > 0:
    for i in indices[0]:
        # Asegurarse de que el índice esté dentro del rango de chunks y chunk_sources
        if 0 <= i < len(chunks) and 0 <= i < len(chunk_sources):
            print("🔹 Source:", chunk_sources[i]) # Usar chunk_sources para el nombre del archivo
            # Limitar la salida para mayor legibilidad
            print(chunks[i]["content"][:500] + "...")
            print("-" * 80)
        else:
            print(f"⚠️ Índice fuera de rango válido para chunks o chunk_sources: {i}") # Manejar índices inválidos
else:
    print("No se encontraron metadatos de chunks (chunk_sources) para mostrar los resultados.")

Resultados de la búsqueda para 'malaria':
🔹 Source: 2D6FBE4_3.3.5 - Neglected tropical diseases/170_2D6FBE4_Metadata_2024-01-08.csv
a subset of the larger number of people at risk. Mass treatment is limited to those living in districts above a threshold level of prevalence; it does not include all people living in districts with any risk of infection. Individual treatment and care is for those who are or have already been infected; it does not include all contacts and others at risk of infection. This number can better be interpreted as the number of people at a level of risk requiring medical intervention – that is, treatme...
--------------------------------------------------------------------------------
🔹 Source: 442CEA8_3.3.3 - Malaria/170_442CEA8_Dataset_2024-03-08.csv
to DIM_TIME.: YEAR | Temporal dimension value (e.g. 2020). Related to DIM_TIME_TYPE.: 2016 | RATE_PER_1000_N: 10.95893 | RATE_PER_1000_NL: 8.33429 | RATE_PER_1000_NU: 13.66553 Indicator Unique ID (seven-digit UID).

https://medium.com/@myscale/how-to-build-a-rag-powered-chatbot-with-google-gemini-and-myscaledb-79c0024cd237

# Configuración del modelo generativo y Generación de respuesta

Ahora que podemos recuperar información relevante de nuestra base de datos vectorial, el siguiente paso es utilizar un modelo de lenguaje grande (LLM) para generar una respuesta basada en la consulta del usuario y la información recuperada.

Utilizaremos el modelo `gemini-pro` para esta tarea.

El proceso general será:

1.  El usuario hace una consulta.
2.  Vectorizamos la consulta usando la función `get_embeddings`.
3.  Buscamos los chunks más relevantes en el índice FAISS usando el vector de la consulta.
4.  Tomamos el contenido de los chunks recuperados.
5.  Enviamos la consulta original del usuario y el contenido de los chunks recuperados al modelo `gemini-pro` con una instrucción (prompt) para que genere una respuesta informada por esos documentos.
6.  Presentamos la respuesta generada al usuario.

In [28]:
# Configurar el modelo generativo Gemini

model_generative = genai.GenerativeModel('gemini-1.5-flash-latest')


def generate_rag_response(query, retrieved_chunks):
    """
    Genera una respuesta a partir de una consulta y los chunks más relevantes,
    usando el modelo generativo Gemini.

    Args:
        query (str): La pregunta del usuario.
        retrieved_chunks (list): Lista de diccionarios, cada uno con la clave 'content'.

    Returns:
        str: Respuesta generada por Gemini basada en el contexto.
    """
    # Combinar chunks en un único contexto con separación visual
    context = "\n---\n".join([chunk['content'] for chunk in retrieved_chunks])

    # Crear prompt para el modelo generativo
    prompt = f"""Usa la siguiente información para responder la pregunta.

Contexto:
{context}

Pregunta:
{query}

Si no encuentras suficiente información en el contexto, responde que no tienes datos relevantes sobre el tema.
"""

    try:
        response = model_generative.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f" Error al generar respuesta con Gemini: {e}")
        return "Lo siento, no pude generar una respuesta en este momento."


# ==========
# FLUJO DE BÚSQUEDA RAG
# ==========

query = "Violencia sexual en Colombia"

if 'get_embeddings' in globals() and 'index' in globals():
    try:
        # Paso 1: Vectorizar consulta
        query_vec = get_embeddings(query)
        query_vec_np = np.array([query_vec])

        # Paso 2: Buscar los k chunks más relevantes
        distances, indices = index.search(query_vec_np, k=5)

        # Paso 3: Recuperar contenido de los chunks
        retrieved_chunks = [chunks[i] for i in indices[0] if 0 <= i < len(chunks)]

        # Paso 4: Generar respuesta
        if retrieved_chunks:
            print("\n Generando respuesta RAG para tu consulta...")
            rag_response = generate_rag_response(query, retrieved_chunks)
            print("\n Respuesta del Chatbot:\n")
            print(rag_response)
        else:
            print("No se encontraron chunks relevantes.")
    except Exception as e:
        print(f"Error durante el proceso de búsqueda RAG: {e}")
else:
    print(" 'get_embeddings' o 'index' no están definidos.")



 Generando respuesta RAG para tu consulta...

 Respuesta del Chatbot:

No tengo datos relevantes sobre violencia sexual en Colombia en el texto proporcionado.  El texto incluye información sobre enfermedades tropicales desatendidas, homicidios y contaminación del aire, pero no aborda la violencia sexual.

